In [1]:
# add path to explore_by_example src folder
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score

# our system imports
from aideme import *

%matplotlib inline

In [2]:
def plot_fscore(metrics):
    df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
    avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
    avg.plot(ylim=[0,1], marker='o')
    plt.show()

In [3]:
# DUMMY DATA
N = int(1e5)
dim = 2
limit = 2 * (0.001)**(1. / dim)
rng = np.random.RandomState(0)
X = rng.uniform(low=-2, high=2, size=(N, dim))  # do not forget to standardize the data. For this distribution, it should be fine without it.
y_subspace = np.vstack([np.abs(X[:, i]) < limit for i in range(dim)]).T.astype('float')  # partial labels (for each subspace)
y = y_subspace.min(axis=1)
labeled_set = LabeledSet(y, y_subspace)

# visualize data distribution
print('selectivity :', 100 * y.sum() / len(y), '%')

#plt.figure(figsize=(10,8))
#plt.scatter(X[:, 0], X[:, 1], s=0.5, c=['b' if lb else 'r' for lb in y])
#plt.show()

selectivity : 0.105 %


# NO FACTORIZATION EXAMPLE

In [4]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    #metric_reached_threshold('tsm', 0.9),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf', gamma='auto'))  # choose a random point
learner = SimpleMargin(C=1024, kernel='rbf')  # choose point closest to SVM decision boundary
#learner = KernelQueryByCommittee(kernel='rbf', sampling='deterministic', n_samples=8, warmup=1000, thin=100, rounding=True)  # version space algorithm
learner = DualSpaceModel(learner, sample_unknown_proba=0.5, mode='positive')  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # 'repeat' specifies how many times to repeat the exploration process

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.0025483890540622537, tsm: 0.0
iter: 11, fscore: 0.013355380310353599, tsm: 0.0
iter: 21, fscore: 0.058091286307053944, tsm: 0.0
iter: 31, fscore: 0.0, tsm: 0.0
iter: 41, fscore: 0.3333333333333333, tsm: 0.0012192997735586135
iter: 51, fscore: 0.7472527472527473, tsm: 0.020826220553089792
iter: 61, fscore: 0.941747572815534, tsm: 0.20466321243523317
iter: 71, fscore: 0.9532710280373832, tsm: 0.5209580838323353
iter: 81, fscore: 0.9565217391304348, tsm: 0.7272727272727273
iter: 91, fscore: 0.975609756097561, tsm: 0.8547008547008547
iter: 101, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.003804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.047537,"[29799, 22959]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.003603,0.043934,0.002548,0.0,0.444707
2,exploration,0.036767,[48482],[0.0],"[[0.0, 0.0]]",0.001006,0.035761,NaN,NaN,NaN
3,exploration,0.037679,[75194],[0.0],"[[0.0, 0.0]]",0.000847,0.036832,NaN,NaN,NaN
4,exploration,0.038109,[74420],[0.0],"[[0.0, 0.0]]",0.000689,0.037420,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
97,exploration,0.011432,[31320],[1.0],"[[1.0, 1.0]]",0.010097,0.001335,NaN,NaN,NaN
98,exploration,0.770081,[81951],[0.0],"[[0.0, 1.0]]",0.002531,0.767550,NaN,NaN,NaN
99,exploration,0.011581,[30458],[1.0],"[[1.0, 1.0]]",0.010213,0.001368,NaN,NaN,NaN
100,exploration,0.322096,[11103],[0.0],"[[0.0, 1.0]]",0.002531,0.319564,NaN,NaN,NaN


# FACTORIZATION EXAMPLE

In [5]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 100  # number of points to be labeled by the user
SUBSAMPLING = None
INITIAL_SAMPLER = stratified_sampler(labeled_set, pos=1, neg=1)  # start with one random positive sample and one random negative sample
#INITIAL_SAMPLER = random_sampler(10)
CALLBACK = [ # callback functions to be called at the end of each iteration
    classification_metrics(y, 'fscore'), 
    three_set_metric,
]
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
CONVERGENCE_CRITERIA = [
    max_iter_reached(NUMBER_OF_ITERATIONS),
    #all_points_are_known,
    #metric_reached_threshold('fscore', 0.8),
    metric_reached_threshold('tsm', 1.0),
]

explore = PoolBasedExploration(INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT, CONVERGENCE_CRITERIA)

# Factorized AL algorithms
PARTITION = [[i] for i in range(dim)]  #[[0], [1]]

# FACTORIZED VERSION SPACE
# label_function = 'AND', 'OR', 'PROD'
# loss = 'GREEDY', 'SQUARED', 'PRODUCT'
#learner = SubspatialVersionSpace(warmup=100, thin=10, n_samples=8, rounding=True, kernel='rbf', gamma=None, partition=PARTITION, label_function='AND', loss='GREEDY')

# FACTORIZED SIMPLE MARGIN
#learner = SubspatialSimpleMargin(C=1024, kernel='rbf', gamma=5, partition=PARTITION, label_function='AND')

# FACTORIZED DSM
learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), mode='positive', sample_unknown_proba=0.5, partition=PARTITION)  # Dual Space model


# RUN EXPLORATION
metrics = explore.run(X, labeled_set, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
#plot_fscore(metrics)
df = pd.DataFrame.from_dict({i: metric for i, metric in enumerate(metrics[0])}, orient='index')
df

iter: 1, fscore: 0.004549590536851683, tsm: 0.0
iter: 11, fscore: 0.12700841622035194, tsm: 0.0
iter: 21, fscore: 0.26649076517150394, tsm: 0.11849710982658959
iter: 31, fscore: 0.9026548672566371, tsm: 0.7307692307692307
iter: 41, fscore: 0.975609756097561, tsm: 0.9
iter: 51, fscore: 1.0, tsm: 1.0


,phase,iter_time,labeled_indexes,final_labels,partial_labels,fit_time,get_next_time,fscore,tsm,callback_time
0,begin,0.003070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,initial_sampling,0.465899,"[32673, 99310]","[1.0, 0.0]","[[1.0, 1.0], [0.0, 0.0]]",0.432501,0.033398,0.004550,0.000000,0.435433
2,exploration,0.275788,[56192],[0.0],"[[0.0, 0.0]]",0.263946,0.011842,NaN,NaN,NaN
3,exploration,0.106980,[61475],[0.0],"[[0.0, 0.0]]",0.099641,0.007339,NaN,NaN,NaN
4,exploration,0.026425,[86389],[0.0],"[[0.0, 0.0]]",0.019765,0.006659,NaN,NaN,NaN
5,exploration,0.061666,[70065],[0.0],"[[0.0, 0.0]]",0.057059,0.004607,NaN,NaN,NaN
6,exploration,0.046008,[44680],[0.0],"[[0.0, 0.0]]",0.042993,0.003015,NaN,NaN,NaN
7,exploration,0.009364,[73417],[0.0],"[[0.0, 1.0]]",0.006227,0.003137,NaN,NaN,NaN
8,exploration,0.061670,[97678],[0.0],"[[0.0, 0.0]]",0.023358,0.038311,NaN,NaN,NaN
9,exploration,0.012049,[79366],[0.0],"[[0.0, 0.0]]",0.010516,0.001533,NaN,NaN,NaN
